In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from tensorflow import keras
from tqdm import tqdm

In [2]:
interactions_df = pd.read_csv('data_original/interactions.csv')
users_df = pd.read_csv('data_original/users.csv')
items_df = pd.read_csv('data_original/items.csv')

In [3]:
items_df = items_df.rename(columns = {'id' : 'item_id'})

In [4]:
item_cat_feats = ['content_type',
                  'for_kids', 'age_rating', 
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1) 
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

In [5]:
items_ohe_df

,item_id,content_type_film,content_type_series,for_kids_0.0,for_kids_1.0,age_rating_0.0,age_rating_6.0,age_rating_12.0,age_rating_16.0,age_rating_18.0,...,directors_Яннике Систад Якобсен,directors_Янус Мец,directors_Ярив Хоровиц,directors_Ярон Зильберман,directors_Ярополк Лапшин,directors_Ярослав Лупий,"directors_Ярроу Чейни, Скотт Моужер",directors_Ясина Сезар,directors_Ясуоми Умэцу,directors_сения Завьялова
0,10711,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2508,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,10716,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,7868,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,16268,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
15959,2367,False,True,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
15960,10632,False,True,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
15961,4538,False,True,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [6]:
interactions_df = interactions_df[interactions_df.watched_pct > 10]
valid_users = interactions_df['user_id'].value_counts()[interactions_df['user_id'].value_counts() > 10].index
valid_items = interactions_df['item_id'].value_counts()[interactions_df['item_id'].value_counts() > 10].index
interactions_df = interactions_df[interactions_df['user_id'].isin(valid_users) & interactions_df['item_id'].isin(valid_items)]

In [7]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65974
6901


In [8]:
common_users = interactions_df['user_id'].unique()
common_items = interactions_df['item_id'].unique()

interactions_df = interactions_df[interactions_df['item_id'].isin(common_items) & interactions_df['user_id'].isin(common_users)]
items_ohe_df = items_ohe_df[items_ohe_df['item_id'].isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df['user_id'].isin(common_users)]
interactions_df["uid"] = interactions_df["user_id"].astype("category").cat.codes
interactions_df["iid"] = interactions_df["item_id"].astype("category").cat.codes

In [9]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), 
                             interactions_df.iid.nunique())) 

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [10]:
res

array([45., 29., 17., ..., 95., 59., 23.])

In [12]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [13]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    neg_dist = K.sum(K.square(anchor - negative), axis=1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)
 
    return loss

# Используем более качество сэмплирования

In [31]:
import random

def weighted_sampling(weights, n_samples):
    indices = list(range(len(weights)))
    sampled_indices = random.choices(indices, weights=weights, k=n_samples)
    return sampled_indices


quality_metric = interactions_df['watched_pct'] 
weights = quality_metric / quality_metric.sum() 


n_samples = 10
sampled_indices = weighted_sampling(weights, n_samples)

# Get the corresponding elements from interactions_vec
sampled_elements = interactions_vec.flatten()[sampled_indices]

In [32]:
def generator(items, users, interactions, weights, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0]-1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1])) % len(items)  # Updated line
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]

In [33]:
gen = generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                users=users_ohe_df.drop(["user_id"], axis=1), 
                interactions=interactions_vec,
                weights=weights)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 16)
вектор взаимодействий юзера с айтемами: (1024, 6897)
вектор 'хорошего' айтема: (1024, 8708)
вектор 'плохого' айтема: (1024, 8708)

вектор фичей юзера: (1024, 16)
вектор взаимодействий юзера с айтемами: (1024, 6897)


In [34]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], ) 
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (8708,)
USER_META_MODEL_SHAPE: (16,)
USER_INTERACTION_MODEL_SHAPE: (6897,)


In [42]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    
    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя, 
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)
    
    add = keras.layers.Add()([layer_1, layer_2])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)
    
    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)
    

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])
    
    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)
    
    return keras.models.Model([inp_meta, inp_interaction], out)

# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [43]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")

In [55]:
opt = keras.optimizers.Adam(lr=0.0005)
model.compile(loss=triplet_loss, optimizer=opt)

In [56]:
with tf.device("/device:GPU:0"):
    model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1),
                      users=users_ohe_df.drop(["user_id"], axis=1),
                      interactions=interactions_vec,
                      batch_size=64,weights=weights),
              steps_per_epoch=100,
              epochs=30,
              initial_epoch=0,
              callbacks=[decay, t_board, check]
              )

Epoch 1/30
100/100 [==============================] - 25s 250ms/step - loss: 0.2050 - lr: 5.0000e-04
Epoch 2/30
100/100 [==============================] - 26s 266ms/step - loss: 0.1973 - lr: 5.0000e-04
Epoch 3/30
100/100 [==============================] - 27s 271ms/step - loss: 0.1866 - lr: 5.0000e-04
Epoch 4/30
100/100 [==============================] - 25s 255ms/step - loss: 0.1794 - lr: 5.0000e-04
Epoch 5/30
100/100 [==============================] - 26s 259ms/step - loss: 0.1733 - lr: 5.0000e-04
Epoch 6/30
100/100 [==============================] - 26s 262ms/step - loss: 0.1644 - lr: 5.0000e-04
Epoch 7/30
100/100 [==============================] - 25s 254ms/step - loss: 0.1695 - lr: 5.0000e-04
Epoch 8/30
100/100 [==============================] - ETA: 0s - loss: 0.1662
Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
100/100 [==============================] - 25s 248ms/step - loss: 0.1662 - lr: 5.0000e-04
Epoch 9/30
100/100 [==============================

# Получение предсказаний для сервиса

In [58]:
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[3]
user_interaction_vec = interactions_vec[3]
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[3]
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1),
                        np.array(user_interaction_vec).reshape(1, -1)])
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))
distance = euclidean_distances(user_vec, item_vec)

# Расчет расстояний для всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
items_vecs = i2v.predict(items_feats)
dists = euclidean_distances(user_vec, items_vecs)

# Нахождение топ-10 рекомендаций для каждого пользователя
users_meta_feats = users_ohe_df.drop(["user_id"], axis=1)
users_vec = u2v.predict([np.array(users_meta_feats), np.array(interactions_vec)])
dists = euclidean_distances(users_vec, items_vecs)
top10_iids = np.argsort(dists, axis=1)[:, :10]
top10_iids_item = np.vectorize(iid_to_item_id.get)(top10_iids)

# Создание датафрейма с рекомендациями
df_dssm = pd.DataFrame({'user_id': [uid_to_user_id[uid] for uid in range(top10_iids_item.shape[0])]})
df_dssm['item_id'] = top10_iids_item.tolist()
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1
df_dssm = df_dssm.groupby('user_id').agg({'item_id': list}).reset_index()

# Вывод полученного датафрейма
df_dssm.head()

2062/2062 [==============================] - 1s 717us/step


,user_id,item_id
0,2,"[10761, 5411, 9342, 2722, 12743, 7626, 12173, ..."
1,21,"[3734, 5536, 10440, 8636, 7829, 15464, 3784, 3..."
2,53,"[3734, 5536, 10440, 8636, 12915, 4151, 2657, 1..."
3,60,"[3734, 8636, 7829, 3784, 15464, 5536, 10440, 7..."
4,81,"[10440, 5536, 3734, 8636, 4457, 6809, 2657, 15..."


In [63]:
df_dssm.explode('item_id').groupby('user_id').agg({'item_id': list}).to_json('dssm_recs.json')

In [62]:
df_dssm.explode('item_id').groupby('user_id').agg({'item_id': list})

,item_id
user_id,
2,"[10761, 5411, 9342, 2722, 12743, 7626, 12173, ..."
21,"[3734, 5536, 10440, 8636, 7829, 15464, 3784, 3..."
53,"[3734, 5536, 10440, 8636, 12915, 4151, 2657, 1..."
60,"[3734, 8636, 7829, 3784, 15464, 5536, 10440, 7..."
81,"[10440, 5536, 3734, 8636, 4457, 6809, 2657, 15..."
...,...
1097486,"[3734, 10440, 5536, 15464, 8636, 4457, 3784, 1..."
1097489,"[10761, 5411, 13867, 2722, 3888, 11919, 12057,..."
1097508,"[10761, 13867, 11919, 5411, 9342, 12743, 14942..."
